In [1]:
!pip install requests ipfshttpclient

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.8 MB/s eta 0:00:00
  Created wheel for varint: filename=varint-1.0.2-py3-none-any.whl size=1961 sha256=fd14601d534ec8e2b9e49ac81acbee1f2af57d13b9de9c1b8410438caf90ec60
  Stored in directory: /root/.cache/pip/wheels/39/48/5e/33919c52a2a695a512ca394a5308dd12626a40bbcd288de814
Successfully built varint


In [8]:
import requests
import ipfshttpclient
import hashlib

In [3]:
# Your Pinata API key and secret
PINATA_API_KEY = "d5deb29ab66ea973f31a"
PINATA_SECRET_API_KEY = "b7bd1067e790a58ca4798295738d053fe63b27ad7056b7df9dcfe3d513a2d733"

In [30]:
# Function to upload file to IPFS via Pinata
def upload_to_ipfs(file_path):
    headers = {
        "pinata_api_key": PINATA_API_KEY,
        "pinata_secret_api_key": PINATA_SECRET_API_KEY
    }

    files = {'file': open(file_path, 'rb')}
    response = requests.post('https://api.pinata.cloud/pinning/pinFileToIPFS', files=files, headers=headers)
    if response.status_code == 200:
        return response.json()['IpfsHash']
    else:
        return None

In [31]:
file_path = "/content/MyImageTest.docx"
ipfs_hash = upload_to_ipfs(file_path)
print("File uploaded to IPFS with hash:", ipfs_hash)

File uploaded to IPFS with hash: QmS2PpMmxVy6MgBszStHuSaxu2d77HUbKd8goFyyP3JxgA


In [32]:
# Function to retrieve file from IPFS via Pinata
def retrieve_from_ipfs(ipfs_hash):
    response = requests.get(f"https://gateway.pinata.cloud/ipfs/{ipfs_hash}")
    if response.status_code == 200:
        return response.content
    else:
        return None

In [34]:
ipfs_hash = "QmS2PpMmxVy6MgBszStHuSaxu2d77HUbKd8goFyyP3JxgA"
file_content = retrieve_from_ipfs(ipfs_hash)
if file_content:
    with open("TestDownload.docx", "wb") as f:
        f.write(file_content)
    print("File retrieved from IPFS")
else:
    print("File not found on IPFS")

File retrieved from IPFS


In [28]:
def delete_from_ipfs(ipfs_hash):
    headers = {
        "pinata_api_key": PINATA_API_KEY,
        "pinata_secret_api_key": PINATA_SECRET_API_KEY
    }

    response = requests.delete(f"https://api.pinata.cloud/pinning/unpin/{ipfs_hash}", headers=headers)

    if response.status_code == 200:
        return True
    else:
        print("Error:", response.text)
        return False


In [29]:
ipfs_hash = "QmVen68Yp1reQ9c3TzSctfoXR2VPfpRq33CzVKecptFQia"
deleted = delete_from_ipfs(ipfs_hash)
if deleted:
    print("File deleted from IPFS")
else:
    print("File not found on IPFS")

Error: {"error":{"reason":"CURRENT_USER_HAS_NOT_PINNED_CID","details":"The current user has not pinned the cid: QmVen68Yp1reQ9c3TzSctfoXR2VPfpRq33CzVKecptFQia","message":"The current user has not pinned the cid: QmVen68Yp1reQ9c3TzSctfoXR2VPfpRq33CzVKecptFQia"}}
File not found on IPFS


In [ ]:
# Function to update file on IPFS via Pinata
def update_on_ipfs(old_ipfs_hash, new_file_path):
    # First, upload the new version of the file
    new_ipfs_hash = upload_to_ipfs(new_file_path)
    if new_ipfs_hash is None:
        print("Failed to upload new file to IPFS")
        return False

    # Optionally, delete the old version of the file
    delete_from_ipfs(old_ipfs_hash)

    return new_ipfs_hash


In [ ]:
old_ipfs_hash = "QmSiSK5gwjLnz2EmFbfCfnw3c1fc9Z39LePK82NZs4Twru"
new_file_path = "/content/README.md"

updated_ipfs_hash = update_on_ipfs(old_ipfs_hash, new_file_path)
if updated_ipfs_hash:
    print("File updated on IPFS. New IPFS hash:", updated_ipfs_hash)
else:
    print("Failed to update file on IPFS")


File updated on IPFS. New IPFS hash: Qmbin2ZiP68K59QkSB2GTkUsee8u5Qd5vjM2SsNsDfSvfc


In [ ]:
ipfs_hash = "Qmbin2ZiP68K59QkSB2GTkUsee8u5Qd5vjM2SsNsDfSvfc"
file_content = retrieve_from_ipfs(ipfs_hash)
if file_content:
    with open("downloaded_file.md", "wb") as f:
        f.write(file_content)
    print("File retrieved from IPFS")
else:
    print("File not found on IPFS")

File retrieved from IPFS


In [36]:
# Function to calculate the hash of a file
def calculate_hash(file_path):
    hasher = hashlib.sha256()
    with open(file_path, 'rb') as f:
        while True:
            chunk = f.read(4096)
            if not chunk:
                break
            hasher.update(chunk)
    return hasher.hexdigest()

# Function to retrieve file content from IPFS
def get_ipfs_content(ipfs_hash):
    response = requests.get(f"https://ipfs.io/ipfs/{ipfs_hash}")
    if response.status_code == 200:
        return response.content
    else:
        return None

# Function to verify file contents against IPFS
def verify_file_contents(ipfs_hash, local_file_path):
    # Calculate the hash of the local file
    local_file_hash = calculate_hash(local_file_path)

    # Retrieve the content of the file from IPFS
    ipfs_content = get_ipfs_content(ipfs_hash)

    if ipfs_content is None:
        print("Error retrieving file from IPFS.")
        return False

    # Calculate the hash of the content fetched from IPFS
    ipfs_hash_calculated = hashlib.sha256(ipfs_content).hexdigest()

    # Compare the hashes
    if ipfs_hash_calculated == local_file_hash:
        print("File contents match the contents stored on IPFS.")
        return True
    else:
        print("File contents do not match the contents stored on IPFS.")
        return False

# Example usage
if __name__ == "__main__":
    ipfs_hash = "QmS2PpMmxVy6MgBszStHuSaxu2d77HUbKd8goFyyP3JxgA"  # Replace with your IPFS hash
    local_file_path = "/content/MyImageTest.docx"  # Replace with the path to your local file
    verify_file_contents(ipfs_hash, local_file_path)


File contents do not match the contents stored on IPFS.
